In [2]:
import re
import time
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from IPython.display import display, HTML
from selenium.webdriver.common.by import By
from pyproj import Proj, transform, Transformer
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
url = "https://m.nicebizmap.co.kr/analysis/analysisFree"

In [4]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get(url)

In [5]:
button = driver.find_element(By.CSS_SELECTOR, "div#pc_sheet01>div.overlay_scroll>div.pc_bdy>div.middle>ul>li>a") # 검색
button.click()

In [6]:
button = driver.find_elements(By.CSS_SELECTOR, "ul.loca_list.cb>li") # 처음 화면에서 광주 검색
for i in range(len(button)):
    if button[i].text.find("광주광역시") != -1:
        button[i].click()
        break

In [7]:
button = driver.find_elements(By.CSS_SELECTOR, "ul.loca_list.cb>li>a") # 서구 검색
for i in range(len(button)):
    if button[i].text.find("서구") != -1:
        button[i].click()
        break

In [8]:
time.sleep(1)
button = driver.find_elements(By.CSS_SELECTOR, "ul.loca_list.cb>li>a") # 동 선택
for i in range(len(button)):
    if button[i].text.find("광천동") != -1:
        button[i].click()
        break

In [9]:
button = driver.find_elements(By.CSS_SELECTOR, "div.tab_box>ul.pd>li>a") # 유동인구 탭으로 이동
for i in range(len(button)):
    if button[i].text.find("유동인구") != -1:
        button[i].click()
        break

In [10]:
time.sleep(60)
button = driver.find_element(By.CSS_SELECTOR, "a#handle03") # 세부 항목 닫아서 많이 보이게 하기
button.click()

In [11]:
pin = driver.find_elements(By.CSS_SELECTOR, "div") # 검색

pins = []
num1 = None
num2 = None

for div in pin:
    title_attr = div.get_attribute("title")
    style_attr = div.get_attribute("style")

    if title_attr is not None and style_attr:
        if "position: absolute" in style_attr and \
           "overflow: visible" in style_attr and \
           "box-sizing: content-box !important" in style_attr and \
           "cursor: inherit" in style_attr:
            
            if not num1 and not num2 : # num1, 2 데이터가 없으면 => 기준 데이터 만들기
                num1 = int(re.search(r'\d+', div.get_attribute("style").split(";")[4]).group())
                num2 = int(re.search(r'\d+', div.get_attribute("style").split(";")[5]).group())

            
            if num1 and num2 : # 데이터가 있으면 => 기준 데이터랑 px 계산
                div_lft = num1 - int(re.search(r'\d+', div.get_attribute("style").split(";")[4]).group())
                div_top = num2 - int(re.search(r'\d+', div.get_attribute("style").split(";")[5]).group())
            else:
                print("예상하지 못한 변수... 씨ㅂㅏㄹ")
            
            pins.append((div, div_lft, div_top)) ##### 여기서 left top 값도 가져오기

print(f"찾은 div 개수: {len(pins)}")
print(pins[0])

찾은 div 개수: 3003
(<selenium.webdriver.remote.webelement.WebElement (session="69c2b0e891d27d2aed16e673efbf411b", element="f.998E6DA5C76C69DD700619CCC637A891.d.328001D8232DA7CDC36016BD5EC08C4E.e.567")>, 0, 0)


In [12]:
print(pins[0][0].get_attribute("outerHTML"))
# print(pins[0])

<div title="" style="position: absolute; overflow: visible; box-sizing: content-box !important; cursor: inherit; left: 956px; top: 522px;"><img src="/assets/bizmap/images/icons/popul1_tag.svg" style="width: 21px; height: 22px; cursor: pointer;" alt="광주광역시 서구 광천동 1등급 마커"></div>


In [13]:
legend = driver.find_elements(By.CSS_SELECTOR, "span") # 검색
for i in range(len(legend)):
    if legend[i].text.find("m") != -1:
        print(legend[i].text)
        m = int(re.search(r'\d+', legend[i].text).group())
        px = legend[i].size['width']
# meter_per_px = m / px  

300m


In [14]:
# 기준 좌표 (0,0 픽셀 위치)
origin_x = 306851.822
origin_y = 3893055.319

# 픽셀당 실제 거리 (m)
meter_per_px = m / px  # 예: 50 / 100 = 0.5

# UTM 좌표 변환을 위한 설정 (52N 지역)
zone_number = 52
northern = True  # 북반구 여부

utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=not northern)
wgs84_proj = Proj(proj='latlong', datum='WGS84')
transformer = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)

# UTM 좌표 변환 및 저장
utm_coords = []
for pin in pins:
    dx, dy = pin[1], pin[2]
    utm_x = origin_x - dx * meter_per_px  # X축 반전
    utm_y = origin_y + dy * meter_per_px  # Y축 반전
    lon, lat = transformer.transform(utm_x, utm_y)
    utm_coords.append((lon, lat))


# DataFrame 생성 (위도, 경도만 저장)
df = pd.DataFrame(utm_coords, columns=['경도', '위도'])

# CSV 저장
df.to_csv("test.csv", encoding='utf-8-sig', index=False, header=False)

# 결과 출력
df


,경도,위도
0,126.879325,35.161890
1,126.878727,35.161844
2,126.879283,35.161854
3,126.879323,35.161960
4,126.879155,35.161852
...,...,...
2998,126.875375,35.167405
2999,126.876348,35.167809
3000,126.873846,35.166992
3001,126.875707,35.167797
